# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [26]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [27]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [28]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [29]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [30]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)    

#### Set Keyspace

In [31]:
try: 
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

## First Query
---
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
---
### For this query we will create the table _music_app_history_ with partition key _session_id_ and clustering key _item_in_session_. Data is stored and partitioned per session_id and sorted ASC by item_in_session. The partitioning per session_id allows us fast retrieval of data per single session. 

In [33]:
ct_music_app_history = """
CREATE TABLE IF NOT EXISTS music_app_history
(
    session_id int,
    item_in_session int,
    artist text, 
    length float,  
    song text,
    PRIMARY KEY (session_id, item_in_session)
) 
"""

try: 
    session.execute(ct_music_app_history)
except Exception as e:
    print(e)

In [34]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history (session_id, item_in_session, artist, length, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [35]:
query1 = "SELECT artist, song, length FROM music_app_history WHERE session_id=338 AND item_in_session=4"
try: 
    rows = session.execute(query1)
    
except Exception as e:
    print(e)


for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Second Query
---

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

---
### For this query we will create the table _*user_history*_ with partition keys _user_id_ and _session_id and clustering key *item_in_session* . Data is stored and partitioned per user_id and session_id as a composite primary key and sorted ASC by item_in_session. A composite primary key allows us a better partitioning of the data. We have added the clustering key item_in_session so we can stick to the sorting order required in the query. 

In [37]:
ct_user_history = """
CREATE TABLE IF NOT EXISTS user_history
(
user_id int, 
session_id int,
item_in_session int,
artist text,
song text, 
first_name text, 
last_name text,
PRIMARY KEY ((user_id, session_id),item_in_session)
)
"""

try: 
    session.execute(ct_user_history)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_history (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))      

In [38]:
query2= "SELECT artist, song, first_name, last_name FROM user_history WHERE user_id=10 AND session_id=182"

try: 
    rows = session.execute(query2)
    
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)  

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Third Query
---

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

---

### For this query we will create the table _*song_histoy*_ with partition key _song_ and clustering key *user_id*. Data is stored and partitioned per song text, adding user_id as composite key helps use to create a unique entry.


In [39]:
ct_song_histoy = """
CREATE TABLE IF NOT EXISTS song_histoy
(
song text,
user_id int,
first_name text, 
last_name text,
PRIMARY KEY (song, user_id)
)
"""
try: 
    session.execute(ct_song_histoy)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_histoy (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],  int(line[10]), line[1], line[4]))

In [40]:
query3= "SELECT first_name, last_name FROM song_histoy WHERE song='All Hands Against His Own'"

try: 
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [41]:
drop_music_app_history= """DROP TABLE IF EXISTS music_app_history"""
drop_user_history= """DROP TABLE IF EXISTS user_history"""
drop_song_history= """DROP TABLE IF EXISTS song_histoy"""
drop_array = [drop_music_app_history, drop_user_history, drop_song_history]

for drop_query in drop_array:
    try:
        session.execute(drop_query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()

## OPTIONAL: Question for the reviewer
 
If you have any question about the starter code or your own implementation, please add it in the cell below. 

For example, if you want to know why a piece of code is written the way it is, or its function, or alternative ways of implementing the same functionality, or if you want to get feedback on a specific part of your code or get feedback on things you tried but did not work.

Please keep your questions succinct and clear to help the reviewer answer them satisfactorily. 

> **_Your question_**